In [50]:
import torch
import pandas as pd
import torch.nn as nn
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super(LSTM, self).__init__()
    self.M = hidden_dim
    self.L = layer_dim

    self.rnn = nn.LSTM(
        input_size=input_dim,
        hidden_size=hidden_dim,
        num_layers=layer_dim,
        batch_first=True)
    #batch_first to have (batch_dim, seq_dim, feature_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, X):
    # initial hidden state and cell state
    h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
    c0 = torch.zeros(self.L, X.size(0), self.M).to(device)

    out, (hn, cn) = self.rnn(X, (h0.detach(), c0.detach()))

    # h(T) at the final time step
    out = self.fc(out[:, -1, :])
    return out

model = LSTM(12, 512, 2, 1)
model.load_state_dict(torch.load('LSTM_model.pth'))
model.eval()

LSTM(
  (rnn): LSTM(12, 512, num_layers=2, batch_first=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [51]:

#Reading data 
weather = pd.read_csv("Input.csv", on_bad_lines='skip')
weather = weather[['Aasta','Kuu','Päev','Kell (UTC)','Tunni keskmine summaarne kiirgus W/m²','Õhurõhk merepinna kõrgusel hPa','Õhurõhk jaama kõrgusel hPa','Tunni sademete summa mm','Suhteline õhuniiskus %','Õhutemperatuur °C','Tunni miinimum õhutemperatuur °C','Tunni maksimum õhutemperatuur °C','10 minuti keskmine tuule suund °', '10 minuti keskmine tuule kiirus m/s', 'Tunni maksimum tuule kiirus m/s']]

# Combine 'Aasta', 'Kuu', 'Päev' and 'Kell (UTC)' into a datetime
weather['Date Time'] = pd.to_datetime(weather['Aasta'].astype(str) + weather['Kuu'].astype(str).str.zfill(2) + weather['Päev'].astype(str).str.zfill(2) + weather['Kell (UTC)'], format='%Y%m%d%H:%M')
date_time = weather['Date Time']
# Remove the 'Date Time' column
weather = weather.drop(['Aasta', 'Kuu', 'Päev', 'Kell (UTC)'], axis=1)

weather = weather.rename(columns={'Õhutemperatuur °C': 'temperature', 'Date Time': 'date', 'Tunni keskmine summaarne kiirgus W/m²' : 'radiation','Õhurõhk merepinna kõrgusel hPa' : 'pressure','Õhurõhk jaama kõrgusel hPa': 'pressure2','Tunni sademete summa mm' : 'precip','Suhteline õhuniiskus %' : 'humidity','Tunni miinimum õhutemperatuur °C' : 'tempmin','Tunni maksimum õhutemperatuur °C': 'tempmax','10 minuti keskmine tuule suund °' : 'winddir', '10 minuti keskmine tuule kiirus m/s' : 'windspeed', 'Tunni maksimum tuule kiirus m/s': 'windmax'})
weather['date'] = pd.to_datetime(weather['date'])
weather.set_index('date')

weather.dropna(inplace=True)


In [52]:
# initial hidden state and cell state
h0 = torch.zeros((model.L, new_data.size(0), model.M)).to(device)
c0 = torch.zeros((model.L, new_data.size(0), model.M)).to(device)


In [58]:
# Drop rows with empty strings
weather = weather.dropna()

# Remove rows with empty strings
weather = weather[weather.apply(lambda x: x.str.strip() != '', axis=1).all(axis=1)]

# Prepare the input data
input_data = weather[['radiation', 'pressure', 'pressure2', 'precip', 'humidity', 'winddir', 'windspeed', 'windmax', 'temperature']]
input_data = input_data.values.reshape(-1, 1, 12)
input_data = torch.from_numpy(input_data.astype(np.float32)).to(device)

# Make predictions
with torch.no_grad():
    predictions = model(input_data)

# Check if predictions tensor has any elements
if predictions.numel() > 0:
    # Print the predicted temperature
    predicted_temperature = predictions.item()
    print("Predicted Temperature:", predicted_temperature)
else:
    print("No predictions available.")



No predictions available.


C:\Users\krist\AppData\Local\Temp\ipykernel_1244\1228148478.py:5: FutureWarning: 'all' with datetime64 dtypes is deprecated and will raise in a future version. Use (obj != pd.Timestamp(0)).all() instead.
  weather = weather[weather.apply(lambda x: x.str.strip() != '', axis=1).all(axis=1)]
